In [25]:
import pandas as pd

In [26]:
# Reddit Comments
# df = pd.read_csv("C:/Users/caleb/Documents/tmp/ssa/powerQueryDiscordData.csv",encoding = "ISO-8859-1")
df = pd.read_csv("./MakerDAO_comment_combined_cleaned.csv")

In [27]:
df.head()

,author,body,created
0,biba8163,"Yes, you are probably right that there is grea...",2021-01-01 01:54:40
1,jolly_octopus,The definition of security broader than the ex...,2021-01-01 05:26:31
2,Think-notlikedasheep,&gt;Binance coin is pretty obviously a securit...,2021-01-01 06:06:36
3,gremlin0x,So the current large debt system is MakerDAO. ...,2021-01-01 07:02:27
4,freehk_fktheccp,This is not exclusive to Ethereum by any means...,2021-01-01 08:25:34


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   author   3142 non-null   object
 1   body     3142 non-null   object
 2   created  3142 non-null   object
dtypes: object(3)
memory usage: 73.8+ KB


In [29]:
df['date'] = df['created'].apply(lambda x: x.split(' ')[0])
df.sort_values(['date'], inplace=True)
df.head(5)

,author,body,created,date
0,biba8163,"Yes, you are probably right that there is grea...",2021-01-01 01:54:40,2021-01-01
1,jolly_octopus,The definition of security broader than the ex...,2021-01-01 05:26:31,2021-01-01
2,Think-notlikedasheep,&gt;Binance coin is pretty obviously a securit...,2021-01-01 06:06:36,2021-01-01
3,gremlin0x,So the current large debt system is MakerDAO. ...,2021-01-01 07:02:27,2021-01-01
4,freehk_fktheccp,This is not exclusive to Ethereum by any means...,2021-01-01 08:25:34,2021-01-01


In [30]:
#filling na with forward fill. 
df = df.fillna(method='ffill')

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 0 to 1252
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   author   3142 non-null   object
 1   body     3142 non-null   object
 2   created  3142 non-null   object
 3   date     3142 non-null   object
dtypes: object(4)
memory usage: 122.7+ KB


In [32]:
# Preparing Text Data for Sentimental Analysis

import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')

def clean_content(contentInput):
    if type(contentInput) == np.float:
        return ""
    contentInput = contentInput.lower()
    contentInput = re.sub("'", "", contentInput) # to avoid removing contractions in english
    contentInput = re.sub("@[A-Za-z0-9_]+","", contentInput)
    contentInput = re.sub("#[A-Za-z0-9_]+","", contentInput)
    contentInput = re.sub(r'http\S+', '', contentInput)
    contentInput = re.sub('[()!?]', ' ', contentInput)
    contentInput = re.sub('\[.*?\]',' ', contentInput)
    contentInput = re.sub("[^a-z0-9]"," ", contentInput)
    contentInput = contentInput.strip()
    contentInput = contentInput.split()
    contentInput = [w for w in contentInput if not w in stop]
    contentInput = " ".join(word for word in contentInput)
    return contentInput

In [33]:
#find top 5 author
df['author'].value_counts().head(5)
top_5_author = df['author'].value_counts().head(6).index.tolist()


top_5_author = top_5_author[1:6]
print(top_5_author)

top_5_author_dict = {}
for author in top_5_author:
    top_5_author_dict[author] = df[df['author'] == author]['body'].to_list()
top_5_author_dict

['nikola_j', 'coinfeeds-bot', 'samsimon123', 'squarov', 'cryptOwOcurrency']


{'nikola_j': ["Just a quick tip if needed - we have the option to create an instantly leveraged CDP at DeFi Saver which looks something like this:  [https://imgur.com/AQemBKt](https://imgur.com/AQemBKt) \n\nAnd everything done at DeFi Saver in terms of MakerDAO is fully compatible with [Oasis.app](https://Oasis.app) should you ever want to switch between them.\n\nFeel free to @ me for any more info (I'm a member of the DFS team).",
  'You would do that by creating a collateralized debt position in one of the apps for managing positions in the MakerDAO protocol, such as Oasis (https://oasis.app, default Maker app) or DeFi Saver (https://defisaver.com, a 3rd party app with extra features).',
  'Heya, if you\'re talking specifically about transferring a Maker Vault once it\'s created - yes, you can do that.\n\nSee the "Transfer" option in the top right of our Maker dashboard at DeFi Saver once you login: [https://app.defisaver.com/makerdao/](https://app.defisaver.com/makerdao/)',
  "This 

In [34]:
#spliting data into months as for time series sentimental analysis
#Getting start and end date of data

# monthList = pd.date_range('2021-01-01','2022-02-01', 
#               freq='MS').strftime("%m/%Y").tolist()

monthList = list(df['year_month'].unique())


#creating blank dict for later

timeSplitData={}
timeSplitDataSentiment = {}

for month in monthList: 
    timeSplitData[month] = []
    timeSplitDataSentiment[month] = []    


KeyError: 'year_month'

In [ ]:
timeSplitData

In [ ]:
for row in df.iterrows():
    #extracting month data from row

    # dateData = row[1]['date']
    # tmp = dateData.find('/') 

    # monthData = dateData[tmp+1:] #removing day e.g '12/'
    # if len(monthData) == 6:
    #     monthData = "0"+monthData
    
    # if monthData in timeSplitData: # check if data is in range of date we want to analyse

    monthData = row[1]['year_month']
    content = row[1]['body']
    #cleaning Data
    cleaned = clean_content(content)
    #Storing into time split dict
    timeSplitData[monthData].append(cleaned)


In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
# ## VADER testing and understanding

# testText = "really painful see sushi chart"
# negativeTest = sid.polarity_scores(testText)

# testText = "market doing well go up"
# positiveTest = sid.polarity_scores(testText)
# ## compound can be use to net the sentiment.. negative mean bear

# print(negativeTest)
# print(positiveTest)

In [ ]:
## Geting Sentiment Score using VANDER's sentiment model
for month in timeSplitData:
    for content in timeSplitData[month]:
        output = sid.polarity_scores(content)
        timeSplitDataSentiment[month].append(output)


In [ ]:
## Calculating Net Weighted Sentiment Score
timeSplitDataNetScore = {}
for month in monthList:
    timeSplitDataNetScore[month] = 0 
    
for month in timeSplitDataSentiment:
    for output in timeSplitDataSentiment[month]:
        compound = output['compound']
        compound = compound / len(timeSplitDataSentiment[month])
        timeSplitDataNetScore[month] +=compound


In [ ]:
timeSplitDataNetScore

In [ ]:
import matplotlib.pyplot as plt

xAxis = []
yAxis = []

for date in timeSplitDataNetScore:
    xAxis.append(date)
    yAxis.append(timeSplitDataNetScore[date])

plt.plot(xAxis,yAxis, color='red', marker='o')
plt.title('Reddit CryptoCurrency Sentimental Analysis')
plt.xlabel('Month')
plt.xticks(rotation=45)
plt.ylabel('Net Sentiment')
plt.grid(True)
plt.show()